**Table of contents**

* [Task](#task)
* [Naive Bayes](#NB)
    * [Feature function](#ff)
    * [MLE](#MLE)
    * [Evaluation](#eval)

    
**Table of Exercises**


* [Exercise 5-1](#ex5-1) (-/1)
* [Exercise 5-2](#ex5-2) (-/2)
* [Exercise 5-3](#ex5-3) (-/3)
* [Exercise 5-4](#ex5-4) (-/2)
* [Exercise 5-5](#ex5-5) (-/4)



**General notes**

* In this notebook you are expected to use $\LaTeX$. 
* Use python3.
* Use NLTK to read annotated data.
* **Document your code**: TAs are more likely to understand the steps if you document them. If you don't, it's also difficult to give you partial points for exercises that are not completely correct.

After completing this lab you should be able to 

* develop Naive Bayes text classifiers
* estimate parameters via MLE
* predict and evaluate models using precision/recall

# <a name="task"> Task

We will be looking into binary sentiment analysis where we have to decide whether a document $x$ (a list of tokens) is positive (class $y=1$) or negative (class $y=0$) towards a subject.

The dataset we will use comes from NLTK [nltk.corpus.sentence_polarity](http://www.nltk.org/howto/corpus.html):

In [ ]:
from nltk.corpus import sentence_polarity

This dataset contains 5331 positive and 5331 negative sentences, which you can obtain as shown below:

In [ ]:
pos_sents = sentence_polarity.sents(categories='pos')
neg_sents = sentence_polarity.sents(categories='neg')
print(len(pos_sents), 'positive sentences such as:\n', ' '.join(pos_sents[0]))
print(len(neg_sents), 'negative sentences such as:\n', ' '.join(neg_sents[0]))

We will use the first 4000 sentences from each class for training, the next 331 for development, and the last 1000 for test:

In [ ]:
training_pos = pos_sents[:4000]
training_neg = neg_sents[:4000]
dev_pos = pos_sents[4000:4331]
dev_neg = neg_sents[4000:4331]
test_pos = pos_sents[4331:]
test_neg = neg_sents[4331:]
print('Training: %d pos and %d neg' % (len(training_pos), len(training_neg)))
print('Development: %d pos and %d neg' % (len(dev_pos), len(dev_neg)))
print('Test: %d pos and %d neg' % (len(test_pos), len(test_neg)))

In [ ]:
training_docs = training_pos + training_neg
dev_docs = dev_pos + dev_neg
test_docs = test_pos + test_neg

# <a name="NB"> Naive Bayes


Feature-rich models are used to model the distribution $P_{Y|X}(y|x)$ of a target variable $y$ conditioned on some high-dimensional data $x$.

One way of doing it is to summarise aspects of $x$ that are relevant to the problem by means of a feature function which returns a vector in some subset of $\mathbb R^D$. For example, this feature function may retain sentiment words in $x$ or some other important aspects of the input. Then instead of modelling $P_{Y|X}(y|x)$ we can, for example, model $P_{Y|F_1^n}(y|f_1^n)$ where we condition on a collection of $n$ features instead.

Conditioning on features of the input, rather than the input directly, does not address the problem on its own, that is, the conditioning context remains high-dimensional. But here is where we can use probability calculus and independence assumptions to make our task simpler.

We can use Bayes rule to invert this conditional:

\begin{align}
(1) \quad P_{Y|F_1^n}(y|f_1^n) = \frac{P_Y(y)P_{F_1^n|Y}(f_1^n|y)}{P_{F_1^n}(f_1^n)}
\end{align}

Now note that the numerator has a conditional where the high-dimensional feature representation of the input is modelled from the target class. That is a problem we can address by making conditional independence assumptions. In particular, by making $F_i$ independent on every other $F_j$ with $i \neq j$ given the target label $y$ we can simplify the problem a lot. We denote this by $F_i \perp F_j \mid y$ for $i\neq j$. Equation (2) shows the resulting model:

\begin{align}
(2) \quad P_{Y|F_1^n}(y|f_1^n) \overset{\text{ind}}{=} \frac{P_Y(y)\prod_{i=1}^n P_{F|Y}(f_i|y)}{P_{F_1^n}(f_1^n)}
\end{align}


Note that we have to model fairly small cpds now:
* a *prior* distribution over classes $P_Y(y)$ 
* a set of cpds $P_{F|Y}$, one per class, over the possible features (these distributions are also called likelihoods, but do not confuse it with the *likelihood function* which is a function of parameters of a statistical model for fixed data)
* the denominator can be inferred by marginalisation, see Equation (3)

\begin{align}
(2) \quad P_{F_1^n}(f_1^n) = \sum_{y \in \mathcal Y} P_{YF_1^n}(y, f_1^n) = \sum_{y \in \mathcal Y} P_{Y}(y)P_{F_1^n|Y}(f_1^n|y) = \sum_{y \in \mathcal Y} P_{Y}(y) \prod_{i=1}^n P_{F|Y}(f_i|y)
\end{align}



## <a name="ff"> Feature function

An important part of a feature-rich model such as Naive Bayes (NB) classifiers is the *feature function*. Here we will develop one. 

In NB classification, features are themselves random variables defined over a certain set $\mathcal F$. We need to first determine this set. In this notebook we will focus on *unigram features*, that is, features defined at the token level.

We will take every token that occurs more than a pre-specified number of times as a potential feature.

Here is an example of how you can do this:

In [ ]:
from collections import Counter


def make_unigram_feature_set(documents, min_freq=1, mark_negation=False):
    """
    This function goes through a corpus and retains all candidate unigram features
     making a feature set. 
    
    :param documents: all documents, each a list of words
    :param min_freq: minimum frequency of a token for it to be part of the feature set
    :param mark_negation: **IGNORE THIS FOR NOW**
    :returns: unigram feature set
    """
    counter = Counter()
    for doc in documents:
        counter.update(doc)
    features = []
    for f, n in counter.most_common():
        if n >= min_freq:
            features.append(f)
        else:
            break
    return frozenset(features)

<a name="ex5-1" style="color:red">**Exercise 5-1**</a> **[1 points]** Modify `make_unigram_feature_set` to optionally pre-process documents by marking words in the scope of a negation with the suffix `_NEG`. For example,  `I am not sure I like the acting` becomes `I am not sure_NEG I_NEG like_NEG the_NEG acting_NEG`. You can use NLTK support for that, see for example, `nltk.sentiment.util.mark_negation`.

In [ ]:
# You can check python documentation by using the following syntax
from nltk.sentiment import util
util.mark_negation?

In [ ]:
from collections import Counter

def make_unigram_feature_set(documents, min_freq=1, mark_negation=False):
    """
    This function goes through a corpus and retains all candidate unigram features
     making a feature set. Optionally, it can also preprocess the corpus annotating
     with _NEG words that are in the scope of a negation (using NLTK helper functions).
    
    :param documents: all documents, each a list of words
    :param min_freq: minimum frequency of a token for it to be part of the feature set
    :param mark_negation: whether to preprocess the document using NLTK's nltk.sentiment.util.mark_negation
        see the documentation `nltk.sentiment.util.mark_negation?`
    :returns: unigram feature set
    """
    pass
    # ***TYPE YOUR SOLUTION HERE***

In [ ]:
# This is just some helper code for better visualization of examples
def inspect_set(input_set, k=5, neg=False):
    """
    Helper function to inspect a few elements in a set of features
    
    :param input_set: a set of features
    :param k: how many elements to select
    :param neg: return `*_NEG` features only
    :returns: up to k elements 
    """
    selected = set()
    for w in input_set:
        if len(selected) < k:            
            if not neg:
                selected.add(w)
            elif '_NEG' in w:
                selected.add(w)
        else:
            break
    return selected

Here are some of the features (without marking negation):

```python
>>> unigram_features = make_unigram_feature_set(training_docs+dev_docs, min_freq=2)
>>> print(len(unigram_features), 'features such as:\n', inspect_set(unigram_features))
```
```
9059 features such as:
 {'white', 'fearless', 'ear', 'tempting', 'meat'}
```

In [ ]:
unigram_features = make_unigram_feature_set(training_docs+dev_docs, min_freq=2)
print(len(unigram_features), 'features such as:\n', inspect_set(unigram_features))

Here are some of the features with pre-processing negation scope.

```python
>>> unigram_features_with_negation = make_unigram_feature_set(training_docs+dev_docs, min_freq=2, mark_negation=True)
>>> print(len(unigram_features_with_negation), 'features such as:\n', 
      inspect_set(unigram_features_with_negation), 
      '\nand:\n', inspect_set(unigram_features_with_negation, neg=True))
```

```
10143 features such as:
 {'white', 'message_NEG', 'fearless', 'ear', 'tempting'} 
and:
 {'ticket_NEG', 'message_NEG', 'street_NEG', 'determined_NEG', 'stereotype_NEG'}     
```

In [ ]:
unigram_features_with_negation = make_unigram_feature_set(training_docs+dev_docs, min_freq=2, mark_negation=True)
print(len(unigram_features_with_negation), 'features such as:\n', 
      inspect_set(unigram_features_with_negation), 
      '\nand:\n', inspect_set(unigram_features_with_negation, neg=True))

Now that we know which features will form the basis of our classifier, we need to implement a feature function. Here we call it a feature map (as we will be using a python dictionary).

In NB classification only the features that occur in an input matter for classification, thus we use a dictionary that maps features to their values if they occur and not otherwise.

This function should take a document $x$ and produce a dict where `f` (a feature) is either 1 (for binary features) or a count (for count features). For the purpose of readability we like to represent features with strings, for example:

* `contains(like) = 1` means that the input contains the word `like`
* `count(like) = 3` means that the input contains 3 occurrences of the word `like`
* `EMPTY() = 1` means that the input contains no known feature

<a name="ex5-2" style="color:red">**Exercise 5-2**</a> **[2 points]** Read the documentation below and implement the feature function described.

In [ ]:
from collections import defaultdict

def make_feature_map(document, feature_set, 
                     binary=True, 
                     mark_negation=False):
    """
    This function takes a document, possibly pre-processes it by marking words in the scope of negation, 
     and constructs a dict indicating which features in `feature_set` fire. Features may be binary, 
     flagging occurrence, or integer, indicating the number of occurrences.
     If no feature can be extracted, a special feature is fired, namely 'EMPTY()'.
     
    :param document: a list of words
    :param feature_set: set of features we are looking for
    :param binary: whether we are indicating presence or counting features in feature_set
    :param mark_negation: whether we should apply NLTK's mark_negation to document before applying the feature function
    :returns: dict with entries 'contains(f)=1/0' for binary features or 'count(f)=n' for count features
    """
    pass
    # ***TYPE YOUR SOLUTION HERE***

Here are some outputs for you to check your implementation:

```python
>>> make_feature_map(pos_sents[7], unigram_features_with_negation, 
                 binary=True, mark_negation=True)
```
```
defaultdict(float,
            {'contains(.)': 1.0,
             'contains(ever_NEG)': 1.0,
             'contains(good_NEG)': 1.0,
             'contains(has_NEG)': 1.0,
             'contains(hell_NEG)': 1.0,
             'contains(is_NEG)': 1.0,
             'contains(literally_NEG)': 1.0,
             'contains(made_NEG)': 1.0,
             'contains(more_NEG)': 1.0,
             'contains(no)': 1.0,
             'contains(perhaps)': 1.0,
             'contains(picture_NEG)': 1.0,
             'contains(road_NEG)': 1.0,
             'contains(that_NEG)': 1.0,
             'contains(the_NEG)': 1.0,
             'contains(to_NEG)': 1.0,
             'contains(with_NEG)': 1.0})
```
```python
>>> make_feature_map(['AKSJDHAU'], unigram_features_with_negation, 
                 binary=True, mark_negation=True)
```
```
defaultdict(float, {'EMPTY()': 1.0})
```

## <a name="MLE"> MLE

Now you will estimate the cpds involved in NB classification. We use a balanced dataset over two classes (positive and negative), so there's no need to compute $P_Y(y)$, it would simply be $0.5$ per class.

You should simply implement cpds for $P_{F|Y}$, that is, exactly $2$ cpds via MLE and you should use Laplace-$\alpha$ smoothing.

<a name="ex5-3" style="color:red">**Exercise 5-3**</a> **[3 points]** Check the documentation below and complete the code for the NB classifier. You will need to implement

* estimation of cpds $P_{F|Y}$ with Laplace smoothing  (1 point) 
* the `classify` method (2 points)

In [ ]:
import numpy as np

def make_cpd(raw_counts, alpha, v):
    """
    This converts a dictionary of raw counts into a cpd.

    :param raw_counts: dict where a key is a feature and a value is its counts (without pseudo counts)
        this should already include the 'EMPTY()' feature
    :param alpha: how many pseudo counts should we add per observation
    :param v: the size of the feature set (already including the 'EMPTY()' feature)
    :returns: a cpd as a dict where a key is a feature and a value is its smoothed probability
    """
    pass
    # ***TYPE YOUR SOLUTION***

class NaiveBayesClassifier:
    
    def __init__(self, training_pos, training_neg, binary, mark_negation, alpha=0.1, min_freq=2):
        """
        :param training_pos: positive documents
            a document is a list of tokens
        :param training_neg: negative documents
            a document is a list of tokens
        :param binary: whether we are using binary or count features
        :param mark_negation: whether we are pre-processing words in negation scope
        :param alpha: Laplace smooth pseudo count
        :param min_freq: minimum frequency of a token for it to be considered a feature
        """
                
        # Make feature set
        print('Extracting features:')
        feature_set = make_unigram_feature_set(
            training_pos + training_neg,  # we use a concatenation of positive and negative training instances
            min_freq=min_freq, 
            mark_negation=mark_negation)
        
        print(' %d features' % len(feature_set))
                
        # Estimate model: 1/2) count        
        print('MLE: counting')        
        counts = [defaultdict(float), defaultdict(float)]
        for docs, y in [(training_pos, 1), (training_neg, 0)]:
            for doc in docs:  # for each document
                # we extract features
                fmap = make_feature_map(doc, 
                                        feature_set, 
                                        binary=binary, 
                                        mark_negation=mark_negation)
                # and gather counts for the pair (y, f)
                for f, n in fmap.items():
                    counts[y][f] += n  
                                
        # 2/2) Laplace-1 MLE
        #  we put EMPTY() is in the support
        print('MLE: smoothing')
        counts[0]['EMPTY()'] += 0
        counts[1]['EMPTY()'] += 0
        # and compute cpds using Laplace smoothing
        self._cpds = [
            make_cpd(counts[0], alpha, len(feature_set) + 1),  # we add 1 because we want EMPTY() to add towards total
            make_cpd(counts[1], alpha, len(feature_set) + 1)]
        print('MLE: done')
            
        # Store data
        self._binary = binary
        self._mark_negation = mark_negation
        self._alpha = alpha
        self._feature_set = feature_set
            
    def get_log_parameter(self, f, y):
        """Returns log P(f|y)"""
        return np.log(self._cpds[y].get(f, self._cpds[y]['EMPTY()']))
        
    def classify(self, doc):
        """
        This function classifies a document by extracting features <f_1...f_n> for it 
         and then computing 
            log P(<f_1...f_n>|Y=0) and log P(<f_1...f_n>|Y=1)
         and finally picking the best (that is, either Y=0 or Y=1).
        
        :param doc: a list of tokens
        :returns: 0 or 1 (the argmax of log P(<f_1...f_n>|y))
        """
        pass
        # ***TYPE YOUR SOLUTION***


This is how you should use the classifier:

```python
>>> classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=False,
    alpha=1., min_freq=2)
```
```
Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done
```

## <a name="eval"> Evaluation

We evaluate binary classifiers on precision, recall, F1 and accuracy. See [Figure 4.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf) and complete the code below:

<a name="ex5-4" style="color:red">**Exercise 5-4**</a> **[2 points]** Classify all documents in a dev set and compute the quantities in [Figure 4.4](https://web.stanford.edu/~jurafsky/slp3/4.pdf).

In [ ]:
def evaluate_model(classifier, pos_docs, neg_docs):
    """
    :param classifier: an NaiveBayesClassifier object
    :param pos_docs: positive documents
    :param neg_docs: negative documents
    :returns: a dictionary containing the number of
        * true positives
        * true negatives
        * false positives
        * false negatives
     as well as 
        * accuracy
        * precision
        * recall 
        * and [F1](https://en.wikipedia.org/wiki/F1_score)
    """
    pass
    # ***TYPE YOUR SOLUTION***
    

For example, our implementation yields:

```python
>>> classifier1 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=False,
    alpha=1., min_freq=2)
```
```
Extracting features:
 8577 features
MLE: counting
MLE: smoothing
MLE: done
```
```python
>>> dev_metrics1 = evaluate_model(classifier1, dev_pos, dev_neg)
>>> print('Development')
>>> print('TP %d TN %d FP %d FN %d' % (dev_metrics1['TP'], dev_metrics1['TN'], dev_metrics1['FP'], dev_metrics1['FN']))
>>> print('P %.4f R %.4f A %.4f F1 %.4f' % (dev_metrics1['P'], dev_metrics1['R'], dev_metrics1['A'], dev_metrics1['F1']))
```
```
Development
TP 239 TN 268 FP 63 FN 92
P 0.7914 R 0.7221 A 0.7659 F1 0.7551
```
```python
>>> classifier2 = NaiveBayesClassifier(
    training_pos, training_neg, 
    binary=True, mark_negation=True,
    alpha=1., min_freq=2)
```
```
Extracting features:
 9581 features
MLE: counting
MLE: smoothing
MLE: done  
```
```python
>>> dev_metrics2 = evaluate_model(classifier2, dev_pos, dev_neg)
>>> print('Development')
>>> print('TP %d TN %d FP %d FN %d' % (dev_metrics2['TP'], dev_metrics2['TN'], dev_metrics2['FP'], dev_metrics2['FN']))
>>> print('P %.4f R %.4f A %.4f F1 %.4f' % (dev_metrics2['P'], dev_metrics2['R'], dev_metrics2['A'], dev_metrics2['F1']))
```
```
Development
TP 248 TN 273 FP 58 FN 83
P 0.8105 R 0.7492 A 0.7870 F1 0.7786
```

<a name="ex5-5" style="color:red">**Exercise 5-5**</a> **[4 points]** Use the dev set to choose the best configuration of 

* alpha (try values like 0.1, 0.5, 1.)
* and binary vs count

for a model that marks negation and one that does not. 

Then report performance on test set for your best model in each case. 

Points:
* 3 points for the search on dev set
* 1 point for the table of test results